In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style(style='ticks')
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import sys
sys.path.insert(0, '/home/sebi/code/analysis/analysis-scripts/induction_analysis')

from lab.classes.dbclasses import dbExperiment, dbMouse

from scipy.ndimage.filters import gaussian_filter1d

In [229]:
mouse = dbMouse('msa0919_2')

expts = mouse.imagingExperiments()
expt = [x for x in expts
        if x.tSeriesDirectory == '/data/Mohsin/2p/msa0919_2/10_7_19/TSeriesmsa0919_2_10072019_cno_induce-003'][0]

In [238]:
spikes = expt.spikes()
sigs = expt.imagingData()
rois = expt.rois()
rlabels = [x.label for x in rois]
on_frames = (expt.behaviorData()['led_context_pin'][1:, 0] / expt.frame_period()).astype(int)

# Detected Indices
indices = [46, 117, 154, 215, 216, 219, 224, 230, 232, 241, 265, 284, 298]

In [70]:
def norm(s):
    if np.nanmin(s) == np.nanmax(s):
        return s
    return (s - np.nanmin(s)) / (np.nanmax(s) - np.nanmin(s))

In [252]:
# Plot peri-stimulation traces and spikes for each trial
# Choose an idx to plot
idx = indices[12]

fig = plt.figure()
ax = fig.add_subplot(111)

pre = 240
post = 300

for i, on_frame in enumerate(on_frames):
    
    sig = norm(gaussian_filter1d(sigs[idx, :, 0], 1))[on_frame-pre:on_frame+post]
    spike = norm(spikes[idx, :])[on_frame-pre:on_frame+post]

    ax.plot(sig + i, 'r')
    ax.plot(spike + i, 'k')

ax.axvline(x=pre, color='k')
ax.axvline(x=pre+30, color='k')

FigureCanvasNbAgg()

In [207]:
# Plot full trace

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(gaussian_filter1d(sigs[idx, :, 0], 1), 'r')
ax.plot(spikes[idx, :], 'k')
ax.vlines(on_frames, -600, 600, 'b', zorder=100)

FigureCanvasNbAgg()

In [253]:
# Where are the stimulations happening during running

pos = expt.behaviorData(imageSync=True)['treadmillPosition']
pos[np.where(np.diff(pos) < -0.5)] = np.nan

fig = plt.figure();
ax = fig.add_subplot(111)
ax.plot(pos)

ax.vlines(on_frames, 0, 1, 'k', zorder=100)

FigureCanvasNbAgg()